In [2]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
import gym
from collections import deque
import random
env = gym.make('LunarLander-v2')

AttributeError: module 'gym.envs.box2d' has no attribute 'LunarLander'

In [ ]:
#HYPERPARAMETERS

learning_rate = 0.001

epsilon = 0.8
epsilon_decay = 0.995
gamma = 0.95

state_size = env.observation_space.shape
action_size = env.action_space.n

batch_size = 20
mem_size = 1000000

In [ ]:
#DQN Agent
model = Sequential()
model.add(Dense(24, input_shape=(state_size[0],), activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(action_size, activation='linear'))

model.compile(optimizer=tf.train.GradientDescentOptimizer(learning_rate), loss='mse')

D = deque()



In [ ]:
#Learning/ Experience Replay

def experience_replay():
    if len(D) < batch_size:
        return
    
    minibatch = random.sample(D, batch_size)

    inputs = np.zeros((batch_size, state_size[0]))
    targets = np.zeros((batch_size, action_size))

    i = 0
    for state, action, reward, new_state, done in minibatch:
        inputs[i] = np.expand_dims(state, axis=0)

        targets[i] = model.predict(np.expand_dims(state, axis=0))
        Q_sa = model.predict(np.expand_dims(new_state, axis=0))

        if done:
            targets[i, action] = reward
        else:
            targets[i, action] = reward + gamma*np.argmax(Q_sa)
        i += 1
    model.train_on_batch(inputs, targets)


In [ ]:
#Play game and learn
state = env.reset()
done=False

while True:
    env.render()
    
    if np.random.rand() <= epsilon:
        #Explore
        action = env.action_space.sample()
    else:
        #Exploit
        action = np.argmax(model.predict(np.expand_dims(state, axis=0))) #max Q-value among actions
        
    new_state, reward, done, info = env.step(action)
    
    D.append([state, action, reward, new_state, done])
    if len(D) > mem_size:
        D.pop()
    
    experience_replay()
    
    state = new_state
    
    epsilon *= epsilon_decay
    
    if done:
        state = env.reset()
        

In [ ]:
#Play
state = env.reset()
tot_reward = 0
done = False

while not done:
    env.render()
    action = np.argmax(model.predict(np.expand_dims(state, axis=0)))
    new_state, reward, done, info = env.step(action)
    tot_reward += reward

tot_reward